In [8]:
import boto3
import botocore
import pandas as pd
from IPython.display import display, Markdown

s3 = boto3.client('s3')
s3_resource = boto3.resource('s3')

In [9]:
s3 = boto3.client('s3')
buckets = s3.list_buckets()
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  aws-glue-scripts-476680029350-eu-west-1
  aws-glue-temporary-476680029350-eu-west-1
  rapid-dw


In [17]:
def list_buckets(match=''):
    response = s3.list_buckets()
    if match:
        print(f'Existing buckets containing "{match}" string:')
    else:
        print('All existing buckets:')
    for bucket in response['Buckets']:
        if match:
            if match in bucket["Name"]:
                print(f'  {bucket["Name"]}')
        else:
                print(f'  {bucket["Name"]}')
                      
def preview_csv_dataset(bucket, key, rows=10):
    data_source = {
            'Bucket': bucket,
            'Key': key
        }
    # Generate the URL to get Key from Bucket
    url = s3.generate_presigned_url(
        ClientMethod = 'get_object',
        Params = data_source
    )

    data = pd.read_csv(url, nrows=rows)
    return data

In [20]:
list_buckets(match='rapid')
list_buckets();

Existing buckets containing "rapid" string:
  rapid-dw
All existing buckets:
  aws-glue-scripts-476680029350-eu-west-1
  aws-glue-temporary-476680029350-eu-west-1
  rapid-dw


In [25]:
bucket='rapid-dw'
bucket_resource = s3_resource.Bucket(bucket)
for file in bucket_resource.objects.all():
    print(f'{file.key}')

2020-10-08_Gesamt/POCTIKWH.EBKVSD
2020-10-08_Gesamt/POCTIKWH.MAKT.G
2020-10-08_Gesamt/POCTIKWH.MARA.G
2020-10-08_Gesamt/POCTIKWH.VSFR.G
2020-10-08_Gesamt/POCTIKWH.VSKB.G
2020-10-08_Gesamt/POCTIKWH.VSKL.G
2020-10-08_Gesamt/POCTIKWH.VSKN.G
2020-10-08_Gesamt/POCTIKWH.VSKT.A
2020-10-08_Gesamt/POCTIKWH.VSKT.G
2020-10-08_Gesamt/POCTIKWH.VSSP.G
2020-10-08_Gesamt/POCTIKWH.VSTE.G
2020-10-08_Gesamt/POCTIKWH.VSVA.G
2020-10-08_Gesamt/POCTIKWH.VSVT.A
2020-10-08_Gesamt/POCTIKWH.VSVT.G


In [27]:
bucket='rapid-dw'
file='2020-10-08_Gesamt/POCTIKWH.VSVA.G'
data_source = {
        'Bucket': bucket,
        'Key': file
    }
# Generate the URL to get Key from Bucket
url = s3.generate_presigned_url(
    ClientMethod = 'get_object',
    Params = data_source
)

data = pd.read_csv(url, nrows=10)

HTTPError: HTTP Error 403: Forbidden

In [28]:
def key_exists(bucket, key):
    try:
        s3_resource.Object(bucket, key).load()
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            # The key does not exist.
            return(False)
        else:
            # Something else has gone wrong.
            raise
    else:
        # The key does exist.
        return(True)

def copy_among_buckets(from_bucket, from_key, to_bucket, to_key):
    if not key_exists(to_bucket, to_key):
        s3_resource.meta.client.copy({'Bucket': from_bucket, 'Key': from_key}, 
                                        to_bucket, to_key)        
        print(f'File {to_key} saved to S3 bucket {to_bucket}')
    else:
        print(f'File {to_key} already exists in S3 bucket {to_bucket}') 

In [30]:
key_exists('rapid-dw','2020-10-08_Gesamt/POCTIKWH.VSFR.G')

ClientError: An error occurred (403) when calling the HeadObject operation: Forbidden